# Oscillating Rod シミュレーションの移行

In [45]:
import boto3
import sagemaker
from sagemaker import get_execution_role

ecr_namespace = 'sagemaker-pysph-test/'
prefix = 'pysph-example-container'

ecr_repository_name = ecr_namespace + prefix
role = get_execution_role()
account_id = role.split(':')[4]
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()

print(account_id)
print(region)
print(role)
print(bucket)

682741310571
us-east-2
arn:aws:iam::682741310571:role/service-role/AmazonSageMaker-ExecutionRole-20190415T123416
sagemaker-us-east-2-682741310571


## 使用するファイルの内容を表示
pygmentize: ファイル内容をハイライト付きで表示する

## build and pushの実行

- 実行権限がない場合
```
!chmod +x ../scripts/build_and_push.sh
```
- Permission Denied防止
```
!chmod +x ../docker/code/train
```

In [46]:
!chmod +x ../docker/code/train
! ../scripts/build_and_push.sh $account_id $region $ecr_repository_name

Sending build context to Docker daemon  79.36kB
Step 1/22 : FROM nvidia/opencl:devel-ubuntu18.04
 ---> fd671294c1b7
Step 2/22 : LABEL maintainer="Sasaki Y"
 ---> Using cache
 ---> 6d4917b308fd
Step 3/22 : ENV DEBIAN_FRONTEND=noninteractive
 ---> Using cache
 ---> fbfdac55da99
Step 4/22 : ARG PYTHON=python3
 ---> Using cache
 ---> 8fc7edf6a2c5
Step 5/22 : ARG PYTHON_PIP=python3-pip
 ---> Using cache
 ---> 92719e58e40b
Step 6/22 : ARG PIP=pip3
 ---> Using cache
 ---> 50ef21aabef2
Step 7/22 : ARG PYTHON_VERSION=3.6.6
 ---> Using cache
 ---> 927be79e7c9d
Step 8/22 : RUN apt-get update && apt-get install -y --no-install-recommends software-properties-common &&     add-apt-repository ppa:deadsnakes/ppa -y &&     apt-get update && apt-get install -y --no-install-recommends         build-essential         ca-certificates         curl         wget         git         libopencv-dev         openssh-client         openssh-server         vim         zlib1g-dev &&     rm -rf /var/lib/apt/lists/*
 --

### ECR上にアップロードしたコンテナのアドレス

In [47]:
container_image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest'.format(account_id, region, ecr_repository_name)
print(container_image_uri)

682741310571.dkr.ecr.us-east-2.amazonaws.com/sagemaker-pysph-test/pysph-example-container:latest


## ハイパーパラメータの設定

In [48]:
! echo "val1, val2, val3" > dummy.csv
print(sagemaker_session.upload_data('dummy.csv', bucket, prefix + '/train'))
print(sagemaker_session.upload_data('dummy.csv', bucket, prefix + '/val'))
! rm dummy.csv

s3://sagemaker-us-east-2-682741310571/pysph-example-container/train/dummy.csv
s3://sagemaker-us-east-2-682741310571/pysph-example-container/val/dummy.csv


## トレーニングの実行

In [52]:
import sagemaker

est = sagemaker.estimator.Estimator(container_image_uri,
                                    role, 
                                    instance_count=1, 
                                    #instance_type='local_gpu', # use local mode
                                    instance_type='ml.p3.2xlarge',
                                    base_job_name=prefix)

est.set_hyperparameters(hp1='value1',
                        hp2=300,
                        hp3=0.001)

train_config = sagemaker.session.TrainingInput('s3://{0}/{1}/train/'.format(bucket, prefix), content_type='text/csv')
val_config = sagemaker.session.TrainingInput('s3://{0}/{1}/val/'.format(bucket, prefix), content_type='text/csv')

job_name=ecr_namespace+"colortry2"
print(job_name)
est.fit({'train': train_config, 'validation': val_config },job_name="colortry2")

sagemaker-pysph-test/color_try2
2020-12-18 07:47:53 Starting - Starting the training job...
2020-12-18 07:48:18 Starting - Launching requested ML instancesProfilerReport-1608277673: InProgress
......
2020-12-18 07:49:19 Starting - Preparing the instances for training......
2020-12-18 07:50:21 Downloading - Downloading input data
2020-12-18 07:50:21 Training - Downloading the training image......
2020-12-18 07:51:21 Training - Training image download completed. Training in progress...Warning: the following symbols are implicitly defined.
  {'j'}
You may want to explicitly declare/define them.
  {'m', 'j', 'k'}
You may want to explicitly declare/define them.
Generating output in /opt/ml/model/experiment
OpenCL code written to /root/.pysph/source/py3.6-linux-x86_64/generated.cl
----------------------------------------------------------------------
No of particles:
  fluid: 14641
  wall: 1488
  Total: 16129
----------------------------------------------------------------------
Setup took: 

対応するS3に
```
output.tar.gz
```
というファイルがあるので、そこからダウンロードして結果を取り出す

In [42]:
est.__dict__

{'image_uri': '682741310571.dkr.ecr.us-east-2.amazonaws.com/sagemaker-pysph-test/pysph-example-container:latest',
 'hyperparam_dict': {'hp1': 'value1', 'hp2': 300, 'hp3': 0.001},
 'role': 'arn:aws:iam::682741310571:role/service-role/AmazonSageMaker-ExecutionRole-20190415T123416',
 'instance_count': 1,
 'instance_type': 'local_gpu',
 'volume_size': 30,
 'volume_kms_key': None,
 'max_run': 86400,
 'input_mode': 'File',
 'tags': None,
 'metric_definitions': None,
 'model_uri': None,
 'model_channel_name': 'model',
 'code_uri': None,
 'code_channel_name': 'code',
 'sagemaker_session': <sagemaker.local.local_session.LocalSession at 0x7f38caec8208>,
 'base_job_name': 'pysph-example-container',
 '_current_job_name': 'pysph-example-container-2020-12-17-06-31-12-414',
 'output_path': 's3://sagemaker-us-east-2-682741310571/',
 'output_kms_key': None,
 'latest_training_job': <sagemaker.estimator._TrainingJob at 0x7f38cab72dd8>,
 'jobs': [<sagemaker.estimator._TrainingJob at 0x7f38cab72dd8>],
 'de